# Joining Wind and Census Data

In [82]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import re 


pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [83]:
path = '/Users/tj/trevorj@berkeley.edu - Google Drive/My Drive/Classes/W210_capstone/W210 Capstone/Data' 
os.chdir(path)

In [84]:
df_census = pd.read_csv('population/ACS/census_bureau_clean/census_bureau.csv')

# df_wind = pd.concat([pd.read_parquet('wind/' + i) for i in os.listdir('wind')])
df_wind = pd.read_parquet('wind/')
# data clean needed for join:
df_wind['ZCTA10'] = df_wind['ZCTA10'].astype(int)
df_wind['year'] = df_wind['year_month'].map(lambda x: int(x[:4]))

In [87]:
df_wind.year.value_counts()

2000    19596
2001    19596
2018    19596
2017    19596
2016    19596
2015    19596
2014    19596
2013    19596
2012    19596
2011    19596
2010    19596
2009    19596
2008    19596
2007    19596
2006    19596
2005    19596
2004    19596
2003    19596
2002    19596
2019    19596
Name: year, dtype: int64

Check zip code overlap rate

In [88]:
census_zips = set(df_census.zip)
wind_zips = set(df_wind.ZCTA10)

print(f"Percent of Census Zips in Wind Data: {len(census_zips & wind_zips) / len(census_zips):.4f}")
print(f"Percent of Wind Zips in Census Data: {len(census_zips & wind_zips) / len(wind_zips):.4f}")

Percent of Census Zips in Wind Data: 0.9231
Percent of Wind Zips in Census Data: 1.0000


Join

In [91]:
df_all = pd.merge(df_wind, df_census, left_on = ['year', 'ZCTA10'], right_on = ['year', 'zip'], how='left')
df_all

,lat,lon,ZCTA10,u,v,wdir,wspd,year_month,year,zip,total_population,total_population_male,total_population_female,population_0_4,population_0_4_male,population_0_4_female,population_5_9,population_5_9_male,population_5_9_female,population_10_14,population_10_14_male,population_10_14_female,population_15_19,population_15_19_male,population_15_19_female
0,37.465,-117.936,89010,0.857699,0.261106,16.931604,0.896562,200001,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35.396,-116.322,89019,0.402479,0.087233,12.229137,0.411824,200001,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.161,-116.139,89060,-0.074703,0.179723,112.570572,0.194630,200001,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,35.957,-115.897,89061,0.637819,-0.031876,357.138916,0.638615,200001,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,39.520,-120.032,89439,0.742497,0.800433,47.150417,1.091786,200001,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391915,39.061,-120.210,96145,0.231725,0.540899,66.809372,0.588445,201912,2019,96145.0,2147.0,1091.0,1056.0,112.0,55.0,57.0,78.0,58.0,20.0,103.0,6.0,97.0,13.0,0.0,13.0
391916,39.149,-120.248,96146,0.231725,0.540899,66.809372,0.588445,201912,2019,96146.0,1145.0,639.0,506.0,57.0,42.0,15.0,64.0,22.0,42.0,38.0,12.0,26.0,23.0,23.0,0.0
391917,39.236,-120.062,96148,0.231725,0.540899,66.809372,0.588445,201912,2019,96148.0,580.0,353.0,227.0,27.0,27.0,0.0,47.0,35.0,12.0,35.0,11.0,24.0,0.0,0.0,0.0
391918,38.732,-120.033,96150,0.035947,0.296003,83.075813,0.298177,201912,2019,96150.0,29327.0,15573.0,13754.0,1582.0,837.0,745.0,1276.0,700.0,576.0,1316.0,785.0,531.0,1453.0,776.0,677.0


Still need to get zip code data for 2000 - 2010, and linear interpolate in between.